In [ ]:
import pandas as pd
import calendar as cal
import datetime
import re
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from matplotlib import font_manager, rc

font_path = "C:/Windows/Fonts/malgun.ttf"
font_name= font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

In [ ]:
path = "../data/"
files = sorted(glob(path+"*.csv"))
files

In [ ]:
# 3. 발전소별 발전량
energy = pd.read_csv(files[2])
# 6. 울산 발전소 동네 날씨 정보
us_fcst = pd.read_csv(files[5])
# 7. 울산 발전소 인근 기상 관측 정보
us_obs = pd.read_csv(files[6])

In [ ]:
print("동서발전 에너지 발전량 정보")
print(energy.info())
print()
print("울산광역시 날씨 관측 정보")
print(us_obs.info())

In [ ]:
# 동서발전 발전소별 발전량 데이터 energy
# 울산 데이터만 뽑기
energy_us = energy.drop(['dangjin_floating', 'dangjin_warehouse', 'dangjin'], axis=1)
energy_us = energy_us.rename(columns={energy_us.columns[0]:'시간',energy_us.columns[1]:'발전량'}, )
# energy_us.head()

# 에너지 데이터 시간 변경
# 타임스탬프 변경
# 24:00:00 시 변경해야함 그러려면
for i in range(len(energy_us)):
    energy_us['시간'][i] = re.sub("24:00:00", "00:00:00", energy_us['시간'][i])
    time = pd.Timestamp(energy_us['시간'][i])
    time = datetime.datetime.strftime(time, '%Y-%m-%d %H:%M')
    energy_us['시간'][i] = time

# 데이터 확인
print(energy_us.loc[22:26])

# 발전량이 0인 데이터 삭제
# energy_us = energy_us[energy_us['발전량'] != 0]
# energy_us.head()

# 파일로 변환하기
energy_us.to_csv('ulsan_energy_df.csv', mode='w', encoding='utf-8-sig', index=False)


In [ ]:
us_energy = pd.read_csv('./ulsan_energy_df.csv')
# us_energy = us_energy[us_energy['발전량'] != 0]
# us_energy.reset_index(inplace=True)/
# us_energy.describe()\
us_energy

In [ ]:
# res = us_energy.groupby(['시간']).apply(lambda x: pd.Series({'집계': len(x['시간'])}))
# # res
# res = us_energy.groupby(['시간'][10:])
# res

In [ ]:
# # 공항 당 항공편 수 확인하기
# result = (
#   flights_df
#   # ORIGIN_AIRPORT 열을 기준으로 데이터를 그룹화한다.
#   # 즉, BOS, SFO, LAX, JFK 의 데이터를 모으는 것이다.
#   .groupby(['ORIGIN_AIRPORT'])
#   # 그룹화 한 것들을 집계한다. (행의 갯수를 계산)
#   .apply(lambda x: pd.Series({
#       'nb_flights': len(x['ORIGIN_AIRPORT'])
#   }))
#   .reset_index()
# )
# result.head()

In [ ]:
us_obs

In [ ]:
us_obs_drop = us_obs.drop(['지점'], axis=1)
us_obs = us_obs_drop.rename(columns={us_obs_drop.columns[1]:"시간"})
us_obs

In [ ]:
# us_obs_energy = pd.merge(us_obs, us_energy, how="left", left_on='시간', right_on='시간')
# us_obs_energy.to_csv('ulsan_obs_energy.csv', mode='w', encoding='utf-8-sig', index=False)
us_obs_energy = pd.read_csv('./ulsan_obs_energy.csv')
us_obs_energy = us_obs_energy[us_obs_energy['발전량'] != 0]
us_obs_energy

In [ ]:
# res = us_obs_energy.groupby(us_obs_energy['시간'].str.contains("18:00")).apply(lambda x: pd.Series({'집계': len(x['시간'].str.contains("00:00"))}))
# res
us_obs_energy.reset_index(inplace=True, drop=True)
us_obs_energy

In [ ]:
us_obs_energy['시간대코드'] = -1
for i in range(len(us_obs_energy)):
    if(us_obs_energy['시간'][i][11:] == ('06:00') or us_obs_energy['시간'][i][11:] == ('07:00') or us_obs_energy['시간'][i][11:] == ('08:00')):
        us_obs_energy['시간대코드'][i] = '6 - 8시'
    elif (us_obs_energy['시간'][i][11:] == ('09:00') or us_obs_energy['시간'][i][11:] == ('10:00') or us_obs_energy['시간'][i][11:] == ('11:00')):
        us_obs_energy['시간대코드'][i] = '9 - 11시'
    elif (us_obs_energy['시간'][i][11:] == ('12:00') or us_obs_energy['시간'][i][11:] == ('13:00') or us_obs_energy['시간'][i][11:] == ('14:00')):
        us_obs_energy['시간대코드'][i] = '12 - 14시'
    elif (us_obs_energy['시간'][i][11:] == ('15:00') or us_obs_energy['시간'][i][11:] == ('16:00') or us_obs_energy['시간'][i][11:] == ('17:00')):
        us_obs_energy['시간대코드'][i] = '15 - 17시'
    elif (us_obs_energy['시간'][i][11:] == ('18:00') or us_obs_energy['시간'][i][11:] == ('19:00') or us_obs_energy['시간'][i][11:] == ('20:00')):
        us_obs_energy['시간대코드'][i] = '18 - 20시'
    else:
        us_obs_energy['시간대코드'] = -1
        print(us_energy[i])   


In [ ]:
us_obs_energy

In [ ]:
# res = us_obs_energy.groupby(['시간대코드']).apply(lambda x: pd.Series({'집계': len(x['시간대코드']), '총발전량': sum(x['발전량']), '총발전량': sum(x['발전량'])}))
res = us_obs_energy.groupby(['시간대코드']).apply(lambda x: pd.Series({'평균': x['발전량'].mean(), '총발전량': sum(x['발전량'])}))
res = us_obs_energy.groupby(['시간대코드']).apply(lambda x: pd.Series({'집계': len(x['시간대코드']), '총발전량': sum(x['발전량']), '총발전량': sum(x['발전량'])}))
res = us_obs_energy.groupby(['시간대코드']).apply(lambda x: pd.Series({'평균': x['발전량'].mean(), '총발전량': sum(x['발전량'])}))
# res.set_index('시간대코드')
# res = res.reset_index()
# res.sort_values(by='시간대코드')
res

In [ ]:
us_obs_energy.describe()

In [ ]:
plt.figure()
res['총발전량'].plot(kind='barh', grid=True, figsize=(5, 5))
plt.show()

In [ ]:
us_obs_energy_test = us_obs_energy.drop(['지점명', '시간'], axis=1)
us_obs_energy_test

In [ ]:
sns.pairplot(us_obs_energy_test, kind="hist", hue="시간대코드")
plt.show()

In [ ]:
plt.rc('font', family="Malgun Gothic")
sns.countplot(x="기온(°C)", hue="시간대코드", data=us_obs_energy)
plt.show()

In [ ]:
sns.heatmap(us_obs_energy_test, annot=True, fmt="d", linewidths=1)
plt.show()